In [1]:
import os

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [14]:
paths['PROTOC_PATH']

'Tensorflow\\protoc'

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [5]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [9]:
# allows to pull model directly from tensorflow model Zoo
!wget --version
if os.name=='nt':
    !pip install wget
    import wget

GNU Wget 1.21.4 built on mingw32.

+cares +digest +gpgme +https +ipv6 +iri +large-file +metalink -nls 
+ntlm +opie +psl +ssl/openssl 

Wgetrc: 
    C:/msys64/qemu/opt/misc-x64/etc/wgetrc (system)
Compile: 
    x86_64-w64-mingw32-gcc -DHAVE_CONFIG_H 
    -DSYSTEM_WGETRC="/opt/misc-x64/etc/wgetrc" 
    -DLOCALEDIR="/opt/misc-x64/share/locale" -I. -I../../src -I../lib 
    -I../../lib -I/opt/misc-x64/include 
    -IC:/msys64/qemu/opt/misc-x64/include 
    -IC:/msys64/qemu/opt/misc-x64/include 
    -IC:/msys64/qemu/opt/misc-x64/include -DCARES_STATICLIB 
    -IC:/msys64/qemu/opt/misc-x64/include -DPCRE2_STATIC 
    -IC:/msys64/qemu/opt/misc-x64/include 
    -IC:/msys64/qemu/opt/misc-x64/include -DHAVE_LIBSSL -I 
    -IC:/msys64/qemu/opt/misc-x64/include -DNDEBUG -ggdb 
    -mtune=broadwell -mtune=znver2 -O2 -pipe 
Link: 
    x86_64-w64-mingw32-gcc -IC:/msys64/qemu/opt/misc-x64/include 
    -IC:/msys64/qemu/opt/misc-x64/include 
    -IC:/msys64/qemu/opt/misc-x64/include -DCARES_STATICLIB 
 

In [ ]:
#cloning tensorflow 
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection 

# Check if the operating system is POSIX-compliant (e.g., Linux, macOS)
if os.name=='posix':  
    # Install the Protocol Buffers compiler on POSIX systems
    !apt-get install protobuf-compiler
    # Change directory to TensorFlow models/research and compile the proto files, 
    # then copy the setup.py file for TensorFlow 2 and install the package
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && \
    cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
# Check if the operating system is Windows (NT)
if os.name=='nt':
    # Define the URL to download the Protocol Buffers compiler for Windows
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    # Download the Protocol Buffers compiler using wget
    wget.download(url)
    # Move the downloaded zip file to the specified path
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    # Change directory to the specified path and extract the zip file
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    # Add the Protocol Buffers compiler's bin directory to the system's PATH environment variable
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    # Change directory to TensorFlow models/research, compile the proto files, copy the setup.py file,
    # and build and install the package
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && \
    copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    # Change directory to TensorFlow models/research/slim and install the package in editable mode
    !cd Tensorflow/models/research/slim && pip install -e .

In [10]:
import google.protobuf

print(google.protobuf.__version__)

3.20.3


In [12]:
!where protoc

C:\AdditionalPackages\protoc\bin\protoc.exe


In [15]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2024-08-14 17:31:00.948874: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-14 17:31:14.335870: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Traceback (most recent call last):
  File "c:\TFOD\Tensorflow\models\research\object_detection\builders\model_builder_tf2_test.py", line 24, in <module>
    from object_detection.builders import model_builder
ModuleNotFoundError: No module named 'object_detection'


In [16]:
# Run Model Bulider Test file, just to verify everything's working properly
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

python: can't open file 'c:\content\models\research\object_detection\builders\model_builder_tf2_test.py': [Errno 2] No such file or directory
